In [ ]:
from dataclasses import dataclass
from typing import Any, Optional, TypedDict
from urllib.parse import urljoin

import requests
from bs4 import BeautifulSoup

ROOT = "https://www.riss.kr/"
PATH = "search/Search.do"
param_dict = {
  "isDetailSearch": 'N',
  "searchGubun": True,
  "viewYn": 'OP',
  "strQuery": '엣지 디바이스',
  "order": '/DESC',
  "onHanja": False,
  "strSort": 'RANK',
  "iStartCount": 0,
  "fsearchMethod": 'search',
  "sflag": 1,
  "isFDetailSearch":'N',
  "pageNumber": 1,
  "icate": 're_a_kor',
  "colName": 're_a_kor',
  "pageScale": 10,
  "isTab": 'Y',
  "query": '엣지 디바이스',
}


HeaderType = TypedDict("HeaderType", {"User-Agent": str, "Referer": str})


@dataclass(frozen=True)
class PageResponseReturnType:
  response: requests.Response
  soup: BeautifulSoup


def get_page_response_with_soup(url: str, *, query_params: Optional[dict[Any, Any]] = None, header: Optional[HeaderType] = None) -> PageResponseReturnType:
  response = requests.get(url, params=query_params, headers=header)
  soup = BeautifulSoup(response.text, "html.parser")


  return PageResponseReturnType(response=response, soup=soup)


paper_list_response = get_page_response_with_soup(urljoin(ROOT, PATH), query_params=param_dict)
previous_link, paper_list_soup = paper_list_response.response.url, paper_list_response.soup

paper_list_selector = ".srchResultListW > ul >  li"
paper_list_element = paper_list_soup.select(paper_list_selector)

len(paper_list_element)


10

In [14]:
for paper_element in paper_list_element:
  title = paper_element.select_one(".title > a").get_text(strip=True)
  link = urljoin(ROOT, paper_element.select_one(".title > a").get("href"))
  print("======== visit link =========")
  paper_detail_response = get_page_response_with_soup(link, header={
    "User-Agent": "Mozilla/5.0",
    "Referer": previous_link
  })
  paper = paper_detail_response.soup
  press = paper.find("span", string="발행기관").find_next_sibling().get_text(strip=True)
  year = paper.find("span", string="발행연도").find_next_sibling().get_text(strip=True)
  keywords = [] if not paper.find("span", string="주제어") else [keyword.strip() for keyword in paper.find("span", string="주제어").find_next_sibling().text.split(";")]

  print(title, link, press, year, keywords, sep="\n")
  print("=========================================================\n")
  

======== visit link =========
IoT엣지디바이스를 이용한 경량화 클라우드 컴퓨팅환경 구축 및 검증
https://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=71d192c1127301b047de9c1710b0298d&keyword=엣지 디바이스
한국산업기술융합학회(구. 산업기술교육훈련학회)
2023
['엣지디바이스', '클라우드컴퓨팅', '사물인터넷', 'Edge Devices', 'Cloud Computing', 'Internet of Things', 'API', 'MQTT']

======== visit link =========
전기차 데이터 수집을 위한엣지디바이스환경 설계
https://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=35a1e5c966b9c0acb36097776a77e665&keyword=엣지 디바이스
한국자동차공학회
2022
['Edge Computing(엣지 컴퓨팅)', 'KubeEdge(쿠베 엣지)', 'V2X Data (차량통신 데이터)', 'xEV Data (전기차 데이터)', 'Virtual Compting(가상 컴퓨팅)']

======== visit link =========
엣지-디바이스협업 환경에서 지연 시간 최소화를 위한 ResNet 동적 분할 전략 연구
https://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=3c518f15de0b5d8a4884a65323211ff0&keyword=엣지 디바이스
한국통신학회
2025
[]

======== visit link =========
딥러닝기반엣지디바이스데이터 압축기술 연구
https://www.riss.kr/search/detail/DetailView.do?p_ma

True

# dataclass vs TypedDict

- TypedDict: dictionary형태의 문법과 추론을 지원하는 타입 흰트 문법
  - 함수형 문법
    - 이중 dictionary의 타입을 클래스형은 추론 못 하지만 함수형 문법은 추론 가능함.
    ```python
    HeaderType = TypedDict("HeaderType", {"User-Agent": str, "Referer": str})
    ```
  - 클래스형 문법 :
    ```python
    class HeaderType(TypedDict):
      User-Agent: str
      Referer: str
    ```

- dataclass
  - 함수형 문법: @dataclass를 이용
  - TypedDict과 다르게 HeaderType(response = ... , soup = ...) 처럼 명시적으로 타입을 지정하여 사용할 수 있음.
  ```python
    @dataclass(frozen=True)
    class PageResponseReturnType:
      response: requests.Response
      soup: BeautifulSoup
  ```
  